# Gets the average number of peaks that intersect skipped exon regions
- spans: | ]----[   ]----[ |
- 2244 native cassette exons for K562: 
    - /projects/ps-yeolab3/encode/k562_brenton-graveley_ambiguous_bams_for_integrated_analysis.txt.nSEall_0.5_20170805.NR
- 1832 native cassette exons for HepG2: 
    - /projects/ps-yeolab3/encode/hepg2_brenton-graveley_ambiguous_bams_for_integrated_analysis.txt.nSEall_0.5_20170805.NR

In [1]:
import pandas as pd
import os
import glob

In [2]:
wd = '/projects/ps-yeolab3/bay001/maps/current/se_peak/'

# peak_hist contains a file with the number of peaks intersecting a region and the number of peaks that don't.
# choose native cassette exons (all skipped exon regions)
peak_hist = glob.glob(os.path.join(wd, '*native-cassette-exons.miso.hist.overlapping_peaks'))

In [5]:
summed = 0 # sum of all peaks intersecting an event region
total_count = 0 # total number of RBPs counted.

for fn in peak_hist:
    df = pd.read_table(fn)
    df['percentage'] = df['intersect'].div((df['intersect'] + df['no_intersect']))
    summed += df.ix[0]['intersect']
    total_count += 1


print("total number of peaks for all rbps: {}".format(summed))
print("total count: {}".format(total_count))

total number of peaks for all rbps: 58488
total count: 348


In [4]:
summed/float(total_count)

168.06896551724137